# Graph Path Algorithms

In [1]:
import neo4j

import pandas as pd

from IPython.display import display

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

# Lab: Neo4j - Shortest Path, A Star, Yen's

## Assume we have a new high speed train rolled out to 9 cities across the USA; the path finding algorithms are directional, so we need to create the relationships in both directions

In [7]:
my_neo4j_wipe_out_database()

query = """

CREATE
  (seattle:Station {name: 'Seattle', latitude: 47.6062, longitude: -122.3321}),
  (berkeley:Station {name: 'Berkeley', latitude: 37.8715, longitude: -122.2730}),
  (losangeles:Station {name: 'Los Angeles', latitude: 34.0522, longitude: -118.2437}),
  (denver:Station {name: 'Denver', latitude: 39.7392, longitude: -104.9903}),
  (dallas:Station {name: 'Dallas', latitude: 32.7767, longitude: -96.7970}),
  (chicago:Station {name: 'Chicago', latitude: 41.8781, longitude: -87.6298}),
  (newyork:Station {name: 'New York', latitude: 40.7128, longitude: -74.0060}),
  (washington:Station {name: 'Washington', latitude: 38.9072, longitude: -77.0369}),
  (miami:Station {name: 'Miami', latitude: 25.7617, longitude: -80.1918}),
  (seattle)-[:TRACK {track_miles: 798}]->(berkeley),
  (berkeley)-[:TRACK {track_miles: 798}]->(seattle),
  (seattle)-[:TRACK {track_miles: 1303}]->(denver),
  (denver)-[:TRACK {track_miles: 1303}]->(seattle),
  (berkeley)-[:TRACK {track_miles: 1240}]->(denver),
  (denver)-[:TRACK {track_miles: 1240}]->(berkeley),
  (berkeley)-[:TRACK {track_miles: 376}]->(losangeles),
  (losangeles)-[:TRACK {track_miles: 376}]->(berkeley),
  (losangeles)-[:TRACK {track_miles: 1436}]->(dallas),
  (dallas)-[:TRACK {track_miles: 1436}]->(losangeles),
  (denver)-[:TRACK {track_miles: 1003}]->(chicago),
  (chicago)-[:TRACK {track_miles: 1003}]->(denver),
  (denver)-[:TRACK {track_miles: 794}]->(dallas),
  (dallas)-[:TRACK {track_miles: 794}]->(denver),
  (chicago)-[:TRACK {track_miles: 794}]->(newyork),
  (newyork)-[:TRACK {track_miles: 794}]->(chicago),
  (dallas)-[:TRACK {track_miles: 1329}]->(washington),
  (washington)-[:TRACK {track_miles: 1329}]->(dallas),
  (newyork)-[:TRACK {track_miles: 226}]->(washington),
  (washington)-[:TRACK {track_miles: 226}]->(newyork),
  (washington)-[:TRACK {track_miles: 1053}]->(miami),
  (miami)-[:TRACK {track_miles: 1053}]->(washington)
  
"""

session.run(query)

In [8]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,Berkeley,[Station]
1,Chicago,[Station]
2,Dallas,[Station]
3,Denver,[Station]
4,Los Angeles,[Station]
5,Miami,[Station]
6,New York,[Station]
7,Seattle,[Station]
8,Washington,[Station]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,Berkeley,[Station],TRACK,Denver,[Station]
1,Berkeley,[Station],TRACK,Los Angeles,[Station]
2,Berkeley,[Station],TRACK,Seattle,[Station]
3,Chicago,[Station],TRACK,Denver,[Station]
4,Chicago,[Station],TRACK,New York,[Station]
5,Dallas,[Station],TRACK,Denver,[Station]
6,Dallas,[Station],TRACK,Los Angeles,[Station]
7,Dallas,[Station],TRACK,Washington,[Station]
8,Denver,[Station],TRACK,Berkeley,[Station]
9,Denver,[Station],TRACK,Chicago,[Station]


-------------------------
  Density: 0.6
-------------------------


## Shortest Path (Dijkstra's algorithm) - find the shortest path between two nodes

In [9]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [10]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

source = "Berkeley"
target = "Miami"

my_neo4j_run_query_pandas(query, source=source, target=target)

,from,to,totalCost,nodes,costs
0,Berkeley,Miami,4194.0,"[Berkeley, Los Angeles, Dallas, Washington, Mi...","[0.0, 376.0, 1812.0, 3141.0, 4194.0]"


In [11]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

source = "Miami"
target = "Berkeley"

my_neo4j_run_query_pandas(query, source=source, target=target)

,from,to,totalCost,nodes,costs
0,Miami,Berkeley,4194.0,"[Miami, Washington, Dallas, Los Angeles, Berke...","[0.0, 1053.0, 2382.0, 3818.0, 4194.0]"


## A Star - variation of Dijkstra; uses a heuristic function; while constructing the path, at each step estimates the cost to the destination using the heuristic;  faster than Dijkstra, but no guarantee of accuracy; in this implementation the heuristic requires a latitude and longitude for each node and uses great circle distance

In [12]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'TRACK', 
                      {nodeProperties: ['latitude', 'longitude'],
                      relationshipProperties: 'track_miles'})

"""
session.run(query)

In [13]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.astar.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      latitudeProperty: 'latitude',
      longitudeProperty: 'longitude',
      relationshipWeightProperty: 'track_miles'
     }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "Berkeley"
target = "Miami"

my_neo4j_run_query_pandas(query, source=source, target=target)


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,Berkeley,Miami,4194.0,"[Berkeley, Los Angeles, Dallas, Washington, Mi...","[0.0, 376.0, 1812.0, 3141.0, 4194.0]"


In [14]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.astar.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      latitudeProperty: 'latitude',
      longitudeProperty: 'longitude',
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "Miami"
target = "Berkeley"

my_neo4j_run_query_pandas(query, source=source, target=target)

,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,Miami,Berkeley,4194.0,"[Miami, Washington, Dallas, Los Angeles, Berke...","[0.0, 1053.0, 2382.0, 3818.0, 4194.0]"


## Yen's - finds k shortest paths between two nodes; for k = 1, it's the same as Dijkstra; for k=2 it returns the shorest and 2nd shorest path; for k = 3, the 3 shortest paths; useful for when we want alternatives 

In [15]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [16]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.yens.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      k: $k,
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "Berkeley"
target = "Miami"
k = 10

my_neo4j_run_query_pandas(query, source=source, target=target, k=k)


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,Berkeley,Miami,4194.0,"[Berkeley, Los Angeles, Dallas, Washington, Mi...","[0.0, 376.0, 1812.0, 3141.0, 4194.0]"
1,1,Berkeley,Miami,4316.0,"[Berkeley, Denver, Chicago, New York, Washingt...","[0.0, 1240.0, 2243.0, 3037.0, 3263.0, 4316.0]"
2,2,Berkeley,Miami,4416.0,"[Berkeley, Denver, Dallas, Washington, Miami]","[0.0, 1240.0, 2034.0, 3363.0, 4416.0]"
3,3,Berkeley,Miami,5177.0,"[Berkeley, Seattle, Denver, Chicago, New York,...","[0.0, 798.0, 2101.0, 3104.0, 3898.0, 4124.0, 5..."
4,4,Berkeley,Miami,5277.0,"[Berkeley, Seattle, Denver, Dallas, Washington...","[0.0, 798.0, 2101.0, 2895.0, 4224.0, 5277.0]"
5,5,Berkeley,Miami,5682.0,"[Berkeley, Los Angeles, Dallas, Denver, Chicag...","[0.0, 376.0, 1812.0, 2606.0, 3609.0, 4403.0, 4..."


In [17]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.yens.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      k: $k,
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "Miami"
target = "Berkeley"
k = 10

my_neo4j_run_query_pandas(query, source=source, target=target, k=k)


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,Miami,Berkeley,4194.0,"[Miami, Washington, Dallas, Los Angeles, Berke...","[0.0, 1053.0, 2382.0, 3818.0, 4194.0]"
1,1,Miami,Berkeley,4316.0,"[Miami, Washington, New York, Chicago, Denver,...","[0.0, 1053.0, 1279.0, 2073.0, 3076.0, 4316.0]"
2,2,Miami,Berkeley,4416.0,"[Miami, Washington, Dallas, Denver, Berkeley]","[0.0, 1053.0, 2382.0, 3176.0, 4416.0]"
3,3,Miami,Berkeley,5177.0,"[Miami, Washington, New York, Chicago, Denver,...","[0.0, 1053.0, 1279.0, 2073.0, 3076.0, 4379.0, ..."
4,4,Miami,Berkeley,5277.0,"[Miami, Washington, Dallas, Denver, Seattle, B...","[0.0, 1053.0, 2382.0, 3176.0, 4479.0, 5277.0]"
5,5,Miami,Berkeley,5682.0,"[Miami, Washington, New York, Chicago, Denver,...","[0.0, 1053.0, 1279.0, 2073.0, 3076.0, 3870.0, ..."


## You try it - find the shortest path using Dijkstra, A Star, and Yen's (for up to 10 paths) for Los Angeles to New York and return New York to Los Angeles; solutions in graph_path_algorithm_solutions

In [18]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [19]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

source = "Los Angeles"
target = "New York"

my_neo4j_run_query_pandas(query, source=source, target=target)

,from,to,totalCost,nodes,costs
0,Los Angeles,New York,2991.0,"[Los Angeles, Dallas, Washington, New York]","[0.0, 1436.0, 2765.0, 2991.0]"


In [20]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

source = "New York"
target = "Los Angeles"

my_neo4j_run_query_pandas(query, source=source, target=target)

,from,to,totalCost,nodes,costs
0,New York,Los Angeles,2991.0,"[New York, Washington, Dallas, Los Angeles]","[0.0, 226.0, 1555.0, 2991.0]"


In [21]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'TRACK', 
                      {nodeProperties: ['latitude', 'longitude'],
                      relationshipProperties: 'track_miles'})

"""
session.run(query)

In [22]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.astar.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      latitudeProperty: 'latitude',
      longitudeProperty: 'longitude',
      relationshipWeightProperty: 'track_miles'
     }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "Los Angeles"
target = "New York"

my_neo4j_run_query_pandas(query, source=source, target=target)


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,Los Angeles,New York,2991.0,"[Los Angeles, Dallas, Washington, New York]","[0.0, 1436.0, 2765.0, 2991.0]"


In [23]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.astar.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      latitudeProperty: 'latitude',
      longitudeProperty: 'longitude',
      relationshipWeightProperty: 'track_miles'
     }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "New York"
target = "Los Angeles"

my_neo4j_run_query_pandas(query, source=source, target=target)


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,New York,Los Angeles,2991.0,"[New York, Washington, Dallas, Los Angeles]","[0.0, 226.0, 1555.0, 2991.0]"


In [24]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [25]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.yens.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      k: $k,
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "Los Angeles"
target = "New York"
k = 10

my_neo4j_run_query_pandas(query, source=source, target=target, k=k)


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,Los Angeles,New York,2991.0,"[Los Angeles, Dallas, Washington, New York]","[0.0, 1436.0, 2765.0, 2991.0]"
1,1,Los Angeles,New York,3413.0,"[Los Angeles, Berkeley, Denver, Chicago, New Y...","[0.0, 376.0, 1616.0, 2619.0, 3413.0]"
2,2,Los Angeles,New York,3965.0,"[Los Angeles, Berkeley, Denver, Dallas, Washin...","[0.0, 376.0, 1616.0, 2410.0, 3739.0, 3965.0]"
3,3,Los Angeles,New York,4027.0,"[Los Angeles, Dallas, Denver, Chicago, New York]","[0.0, 1436.0, 2230.0, 3233.0, 4027.0]"
4,4,Los Angeles,New York,4274.0,"[Los Angeles, Berkeley, Seattle, Denver, Chica...","[0.0, 376.0, 1174.0, 2477.0, 3480.0, 4274.0]"
5,5,Los Angeles,New York,4826.0,"[Los Angeles, Berkeley, Seattle, Denver, Dalla...","[0.0, 376.0, 1174.0, 2477.0, 3271.0, 4600.0, 4..."


In [26]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.yens.stream(
    'ds_graph', 
    { sourceNode: source,
      targetNode: target,
      k: $k,
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "New York"
target = "Los Angeles"
k = 10

my_neo4j_run_query_pandas(query, source=source, target=target, k=k)


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,New York,Los Angeles,2991.0,"[New York, Washington, Dallas, Los Angeles]","[0.0, 226.0, 1555.0, 2991.0]"
1,1,New York,Los Angeles,3413.0,"[New York, Chicago, Denver, Berkeley, Los Ange...","[0.0, 794.0, 1797.0, 3037.0, 3413.0]"
2,2,New York,Los Angeles,3965.0,"[New York, Washington, Dallas, Denver, Berkele...","[0.0, 226.0, 1555.0, 2349.0, 3589.0, 3965.0]"
3,3,New York,Los Angeles,4027.0,"[New York, Chicago, Denver, Dallas, Los Angeles]","[0.0, 794.0, 1797.0, 2591.0, 4027.0]"
4,4,New York,Los Angeles,4274.0,"[New York, Chicago, Denver, Seattle, Berkeley,...","[0.0, 794.0, 1797.0, 3100.0, 3898.0, 4274.0]"
5,5,New York,Los Angeles,4826.0,"[New York, Washington, Dallas, Denver, Seattle...","[0.0, 226.0, 1555.0, 2349.0, 3652.0, 4450.0, 4..."


# Lab: Neo4j - All Pairs Shortest Path, Single Source Shortest Path

## All Pairs Shorest Path - find the shortest path between all pairs of nodes; this implementation is a bit strange in that it only gives the path cost and not the actual path; 

In [27]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [28]:
query = """

CALL gds.allShortestPaths.stream('ds_graph', 
                                 {relationshipWeightProperty: 'track_miles'}
                                )
YIELD sourceNodeId, targetNodeId, distance
WITH sourceNodeId, targetNodeId, distance
WHERE gds.util.isFinite(distance) = true

WITH gds.util.asNode(sourceNodeId) AS source, gds.util.asNode(targetNodeId) AS target, distance
WHERE source <> target

RETURN source.name AS source, target.name AS target, distance
ORDER BY distance DESC, source.name ASC, target.name ASC

"""

my_neo4j_run_query_pandas(query)


,source,target,distance
0,Miami,Seattle,4379.0
1,Seattle,Miami,4379.0
2,Berkeley,Miami,4194.0
3,Miami,Berkeley,4194.0
4,Los Angeles,Miami,3818.0
...,...,...,...
67,New York,Chicago,794.0
68,Berkeley,Los Angeles,376.0
69,Los Angeles,Berkeley,376.0
70,New York,Washington,226.0


## If we need to get the actual paths, which in most cases we would want, we will need to run Dijkstra multiple times from Python and collect the results 

In [29]:
def my_get_node_list():
    "get a list of nodes in the current graph"
    
    query = "match (n) return n.name as name"
    
    result = session.run(query)
    
    node_list = []
    
    for r in result:
        node_list.append(r["name"])
        
    node_list = sorted(node_list)
    
    return node_list

In [30]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [31]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

nodes = my_get_node_list()

all_pairs_shortest_paths = []

for source in nodes:
    
    for target in nodes:
        
        if source != target:
            
            result = session.run(query, source=source, target=target)
            
            for r in result:

                all_pairs_shortest_paths.append([r["from"], r["to"], r["totalCost"], r["nodes"], r["costs"]])
                
for path in all_pairs_shortest_paths:
    
    print(path)
                      

['Berkeley', 'Chicago', 2243.0, ['Berkeley', 'Denver', 'Chicago'], [0.0, 1240.0, 2243.0]]
['Berkeley', 'Dallas', 1812.0, ['Berkeley', 'Los Angeles', 'Dallas'], [0.0, 376.0, 1812.0]]
['Berkeley', 'Denver', 1240.0, ['Berkeley', 'Denver'], [0.0, 1240.0]]
['Berkeley', 'Los Angeles', 376.0, ['Berkeley', 'Los Angeles'], [0.0, 376.0]]
['Berkeley', 'Miami', 4194.0, ['Berkeley', 'Los Angeles', 'Dallas', 'Washington', 'Miami'], [0.0, 376.0, 1812.0, 3141.0, 4194.0]]
['Berkeley', 'New York', 3037.0, ['Berkeley', 'Denver', 'Chicago', 'New York'], [0.0, 1240.0, 2243.0, 3037.0]]
['Berkeley', 'Seattle', 798.0, ['Berkeley', 'Seattle'], [0.0, 798.0]]
['Berkeley', 'Washington', 3141.0, ['Berkeley', 'Los Angeles', 'Dallas', 'Washington'], [0.0, 376.0, 1812.0, 3141.0]]
['Chicago', 'Berkeley', 2243.0, ['Chicago', 'Denver', 'Berkeley'], [0.0, 1003.0, 2243.0]]
['Chicago', 'Dallas', 1797.0, ['Chicago', 'Denver', 'Dallas'], [0.0, 1003.0, 1797.0]]
['Chicago', 'Denver', 1003.0, ['Chicago', 'Denver'], [0.0, 1003.0

## Single Source Shorest Path - find the shortest paths form one node pairwise with all other nodes; this implementation is a bit strange in that it only gives the path cost and not the actual path;

## Update -  the newer release of this now gives the actual paths

In [32]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [33]:
query = """

MATCH (n:Station {name: $source})
CALL gds.allShortestPaths.delta.stream('ds_graph', 
                                       {sourceNode: n,
                                        relationshipWeightProperty: 'track_miles',
                                        delta: 3.0
                                       }
                                      )
                                        
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs
ORDER BY index

"""

source = "Berkeley"

my_neo4j_run_query_pandas(query, source=source)

,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs
0,0,Berkeley,Seattle,798.0,"[Berkeley, Seattle]","[0.0, 798.0]"
1,1,Berkeley,Berkeley,0.0,[Berkeley],[0.0]
2,2,Berkeley,Los Angeles,376.0,"[Berkeley, Los Angeles]","[0.0, 376.0]"
3,3,Berkeley,Denver,1240.0,"[Berkeley, Denver]","[0.0, 1240.0]"
4,4,Berkeley,Dallas,1812.0,"[Berkeley, Los Angeles, Dallas]","[0.0, 376.0, 1812.0]"
5,5,Berkeley,Chicago,2243.0,"[Berkeley, Denver, Chicago]","[0.0, 1240.0, 2243.0]"
6,6,Berkeley,New York,3037.0,"[Berkeley, Denver, Chicago, New York]","[0.0, 1240.0, 2243.0, 3037.0]"
7,7,Berkeley,Washington,3141.0,"[Berkeley, Los Angeles, Dallas, Washington]","[0.0, 376.0, 1812.0, 3141.0]"
8,8,Berkeley,Miami,4194.0,"[Berkeley, Los Angeles, Dallas, Washington, Mi...","[0.0, 376.0, 1812.0, 3141.0, 4194.0]"


## If we need to get the actual paths, which in most cases we would want, we will need to run Dijkstra multiple times from Python and collect the results 

In [34]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [35]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

nodes = my_get_node_list()

single_source_shortest_paths = []

source = "Berkeley"
    
for target in nodes:

    if source != target:

        result = session.run(query, source=source, target=target)

        for r in result:

            single_source_shortest_paths.append([r["from"], r["to"], r["totalCost"], r["nodes"], r["costs"]])
                
for path in single_source_shortest_paths:
    
    print(path)
                      

['Berkeley', 'Chicago', 2243.0, ['Berkeley', 'Denver', 'Chicago'], [0.0, 1240.0, 2243.0]]
['Berkeley', 'Dallas', 1812.0, ['Berkeley', 'Los Angeles', 'Dallas'], [0.0, 376.0, 1812.0]]
['Berkeley', 'Denver', 1240.0, ['Berkeley', 'Denver'], [0.0, 1240.0]]
['Berkeley', 'Los Angeles', 376.0, ['Berkeley', 'Los Angeles'], [0.0, 376.0]]
['Berkeley', 'Miami', 4194.0, ['Berkeley', 'Los Angeles', 'Dallas', 'Washington', 'Miami'], [0.0, 376.0, 1812.0, 3141.0, 4194.0]]
['Berkeley', 'New York', 3037.0, ['Berkeley', 'Denver', 'Chicago', 'New York'], [0.0, 1240.0, 2243.0, 3037.0]]
['Berkeley', 'Seattle', 798.0, ['Berkeley', 'Seattle'], [0.0, 798.0]]
['Berkeley', 'Washington', 3141.0, ['Berkeley', 'Los Angeles', 'Dallas', 'Washington'], [0.0, 376.0, 1812.0, 3141.0]]


## You try it - find the single source shortest paths from Los Angeles; solutions in graph_path_algorithms_solutions

In [36]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'track_miles'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

nodes = my_get_node_list()

single_source_shortest_paths = []

source = "Los Angeles"
    
for target in nodes:

    if source != target:

        result = session.run(query, source=source, target=target)

        for r in result:

            single_source_shortest_paths.append([r["from"], r["to"], r["totalCost"], r["nodes"], r["costs"]])
                
for path in single_source_shortest_paths:
    
    print(path)
                  
    

['Los Angeles', 'Berkeley', 376.0, ['Los Angeles', 'Berkeley'], [0.0, 376.0]]
['Los Angeles', 'Chicago', 2619.0, ['Los Angeles', 'Berkeley', 'Denver', 'Chicago'], [0.0, 376.0, 1616.0, 2619.0]]
['Los Angeles', 'Dallas', 1436.0, ['Los Angeles', 'Dallas'], [0.0, 1436.0]]
['Los Angeles', 'Denver', 1616.0, ['Los Angeles', 'Berkeley', 'Denver'], [0.0, 376.0, 1616.0]]
['Los Angeles', 'Miami', 3818.0, ['Los Angeles', 'Dallas', 'Washington', 'Miami'], [0.0, 1436.0, 2765.0, 3818.0]]
['Los Angeles', 'New York', 2991.0, ['Los Angeles', 'Dallas', 'Washington', 'New York'], [0.0, 1436.0, 2765.0, 2991.0]]
['Los Angeles', 'Seattle', 1174.0, ['Los Angeles', 'Berkeley', 'Seattle'], [0.0, 376.0, 1174.0]]
['Los Angeles', 'Washington', 2765.0, ['Los Angeles', 'Dallas', 'Washington'], [0.0, 1436.0, 2765.0]]


# Lab: Neo4j - Minimum Spanning Tree

## Minimum Spanning Tree - find a tree structure path that will visit all nodes in the graph with smallest cost; the algorithm actually writes new relations to the graph for the MST

In [37]:
def my_neo4j_wipe_out_mst_relationships():
    "wipe out mst relationships"
    
    query = "match (node)-[relationship:MST]->() delete relationship"
    session.run(query)

In [38]:
my_neo4j_wipe_out_mst_relationships()

In [39]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 
                        {
                            TRACK: {
                                properties: 'track_miles',
                                orientation: 'UNDIRECTED'
                            }
                        }
                       )

"""

session.run(query)

In [40]:
query = """
MATCH (n:Station {name: $source})
CALL gds.spanningTree.write('ds_graph',
                            {
                                sourceNode: n,
                                relationshipWeightProperty: 'track_miles',
                                writeProperty: 'writeCost',
                                writeRelationshipType: 'MST'
                            }
                           )
YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;
"""

source = "Berkeley"

my_neo4j_run_query_pandas(query, source=source)


,preProcessingMillis,computeMillis,writeMillis,effectiveNodeCount
0,0,0,5,9


In [41]:
query = """

MATCH path = (n:Station {name: $source})-[:MST*]-()
WITH relationships(path) AS rels
UNWIND rels AS rel
WITH DISTINCT rel AS rel
RETURN startNode(rel).name AS source, endNode(rel).name AS destination, rel.writeCost AS cost

"""

source = "Berkeley"

my_neo4j_run_query_pandas(query, source=source)

,source,destination,cost
0,Berkeley,Seattle,798.0
1,Berkeley,Los Angeles,376.0
2,Berkeley,Denver,1240.0
3,Denver,Chicago,1003.0
4,Chicago,New York,794.0
5,New York,Washington,226.0
6,Washington,Miami,1053.0
7,Denver,Dallas,794.0


## You try it - find the Minimum Spanning Tree with the source at Los Angeles

In [42]:
my_neo4j_wipe_out_mst_relationships() # Need to remove existing MST

In [43]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 
                        {
                            TRACK: {
                                properties: 'track_miles',
                                orientation: 'UNDIRECTED'
                            }
                        }
                       )
                                
"""

session.run(query)

In [44]:
query = """
MATCH (n:Station {name: $source})
CALL gds.spanningTree.write('ds_graph',
                            {
                                sourceNode: n,
                                relationshipWeightProperty: 'track_miles',
                                writeProperty: 'writeCost',
                                writeRelationshipType: 'MST'
                            }
                           )
YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;
"""

source = "Los Angeles"

my_neo4j_run_query_pandas(query, source=source)

,preProcessingMillis,computeMillis,writeMillis,effectiveNodeCount
0,0,1,5,9


In [45]:
query = """

MATCH path = (n:Station {name: $source})-[:MST*]-()
WITH relationships(path) AS rels
UNWIND rels AS rel
WITH DISTINCT rel AS rel
RETURN startNode(rel).name AS source, endNode(rel).name AS destination, rel.writeCost AS cost

"""

source = "Los Angeles"

my_neo4j_run_query_pandas(query, source=source)

,source,destination,cost
0,Los Angeles,Berkeley,376.0
1,Berkeley,Seattle,798.0
2,Berkeley,Denver,1240.0
3,Denver,Dallas,794.0
4,Denver,Chicago,1003.0
5,Chicago,New York,794.0
6,New York,Washington,226.0
7,Washington,Miami,1053.0


# Lab: Neo4j - Random Walk

## Random Walk - given a path size, randomly walk through nodes until we hit the path size

In [46]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'TRACK', {relationshipProperties: 'track_miles'})"
session.run(query)

In [49]:
query = """

MATCH (home:Station {name: $source})
WITH COLLECT(home) as sourceNodes
CALL gds.randomWalk.stream('ds_graph',
                                {sourceNodes: sourceNodes,
                                 walkLength: $path_size,
                                 walksPerNode: 1,
                                 concurrency: 1
                                }
                               )
YIELD nodeIds
UNWIND nodeIds AS nodeId
RETURN gds.util.asNode(nodeId).name AS node

"""

source = "Berkeley"

path_size = 10

my_neo4j_run_query_pandas(query, source=source, path_size=path_size)

,node
0,Berkeley
1,Denver
2,Chicago
3,New York
4,Washington
5,Miami
6,Washington
7,Dallas
8,Denver
9,Seattle


## Re-run several times, should get different random walks each time

## You try it - take a random walk from the Los Angeles node with path size of 10; solution is in graph_path_algorithms_solutions

In [52]:
query = """

MATCH (home:Station {name: $source})
WITH COLLECT(home) as sourceNodes
CALL gds.randomWalk.stream('ds_graph',
                                {sourceNodes: sourceNodes,
                                 walkLength: $path_size,
                                 walksPerNode: 1,
                                 concurrency: 1
                                }
                               )
YIELD nodeIds
UNWIND nodeIds AS nodeId
RETURN gds.util.asNode(nodeId).name AS node

"""

source = "Los Angeles"

path_size = 10

my_neo4j_run_query_pandas(query, source=source, path_size=path_size)

,node
0,Los Angeles
1,Berkeley
2,Seattle
3,Denver
4,Seattle
5,Berkeley
6,Denver
7,Dallas
8,Washington
9,New York
